## Lunar Perigee Calculation



In [1]:
import pandas as pd
import time
from datetime import datetime, timedelta
import math

# Load the CSV file into a DataFrame (make sure the CSV is correctly formatted)
## NCKT_tabulated_t_values contains t_values used to divide each month into 3 segments, so perigee will be detected for each lunar cycle for desired year
t_values_df = pd.read_csv(r"C:\Users\pawoo\OneDrive\Documents\Coastal_Hazards_Lab_Stuff\NCKT\NCKT_tabulated_T_values.csv")
t_values_df.columns = t_values_df.columns.str.strip()

# Enter year here in form "year.month" where month is in fraction of year
# Ex. The end of April in 2041 would be 2041.33
# This will be the final format of the 't1 + year' columns

# change to desired year
input_year = 2025

# Adjust t1, t2, and t3 with the input year
t_values_df['t1 + year'] = t_values_df['t1'] + input_year
t_values_df['t2 + year'] = t_values_df['t2'] + input_year
t_values_df['t3 + year'] = t_values_df['t3'] + input_year


t_values_df


# K value determines apogee or perigee- removes k values ending in .5 to isolate perigee
# Function to calculate and filter k values
def process_k_values(t_plus_year):
    # Step 1: Calculate initial k
    k = (t_plus_year - 1999.97) * 13.2555

    # Step 2: Round k to nearest integer or 0.5 as per logic
    if abs(k - round(k)) < abs(k - round(k * 2) / 2):
        k = round(k)  # Round to nearest integer
    else:
        k = round(k * 2) / 2  # Round to nearest 0.5
    
    # Step 3: Remove k values ending in .5
    if k % 1 == 0.5:
        return None  # Filter out .5 values
    else:
        return k

# Iterate over each t + year column, process each month, and print results
for col in ['t1 + year', 't2 + year', 't3 + year']:
    t_values_df[f'k_{col}'] =  t_values_df[col].apply(process_k_values)  # Apply function to calculate and filter k

# Select only the columns that contain 'k' values (created earlier in your DataFrame)
k_columns = [f'k_{col}' for col in ['t1 + year', 't2 + year', 't3 + year']]

# Concatenate all the k columns into a single Series, automatically dropping NaNs
k_values_series = pd.concat([t_values_df[col].dropna() for col in k_columns])

# Convert the concatenated Series into a list
k_values_list = k_values_series.tolist()

sorted_list = sorted(k_values_list)




# Function to calculate JDE for a given k value
def calculate_jde(k):
        T = (k / 1325.55)

        
        JDE_initial = 2451534.6698 + (27.55454989 * k) + (-0.0006691 * (T ** 2)) + (-0.000001098 * (T ** 3)) + (0.0000000052 * (T ** 4))

        D = 171.9179 + (335.9106046 * k) - (0.0100383 * T**2) - (0.00001156 * T**3) + (0.000000055 * T**4)
        M = 347.3477 + (27.1577721 * k) - (0.0008130 * T**2) - (0.0000010 * T**3)
        F = 316.6109 + (364.5287911 * k) - (0.0125053 * T**2) - (0.0000148 * T**3)

        # Define the coefficients
        ## Perigee terms: 
        T50 = {
            "Coefficient": [
            -1.6769,
            0.4589,
            -0.1856,
            0.0883,
            (-0.0773 + (0.00019 * T)),
            (0.0502 - (0.00013 * T)),
            -0.0460,
            (0.0422 - (0.00011 * T)),
            -0.0256,
            0.0253,
            0.0237,
            0.0162,
            -0.0145,
            0.0129,
            -0.0112,
            -0.0104,
            0.0086,
            0.0069,
            0.0066,
            -0.0053,
            -0.0052,
            -0.0046,
            -0.0041,
            0.0040,
            0.0032,
            -0.0032,
            0.0031,
            -0.0029,
            0.0027,
            0.0027,
            -0.0027,
            0.0024,
            -0.0021,
            -0.0021,
            -0.0021,
            0.0019,
            -0.0018,
            -0.0014,
            -0.0014,
            -0.0014,
            0.0014,
            -0.0014,
            0.0013,
            0.0013,
            0.0011,
            -0.0011,
            -0.0010,
            -0.0009,
            -0.0008,
            0.0008,
            0.0008,
            0.0007,
            0.0007,
            0.0007,
            -0.0006,
            -0.0006,
            0.0006,
            0.0005,
            0.0005,
            -0.0004,
            ],

            "Argument of Sine": [
                (2 * D),
                (4 * D),
                (6 *D),
                (8 * D),
                ((2 * D) - M),
                (M),
                (10* D),
                ((4 * D) - M), 
                ((6*D) - M),
                (12 * D), 
                (D), 
                ((8 * D) - M),
                (14 * D), 
                (2 * F), 
                (3 *D), 
                ((10 * D) - M),
                (16 * D),
                ((12 * D) - M),
                (5 * D), 
                ((2 * D) + (2 * F)),
                (18 * D), 
                ((14 * D) - M), 
                (7 *D), 
                ((2 * D) + M), 
                (20 * D), 
                (D + M), 
                ((16 * D) - M),
                ((4 * D) + M),
                ( 9 *D),
                ((4 * D )+ (2 * F)),
                ((2 * D) - (2 * M)),
                ((4 * D) - (2 * M)),
                ((6 * D) - (2 * M)),
                (22 * D), 
                ((18 * D) - (M)),
                ((6*D) + M), 
                (11 * D),
                ((8 * D) + M),
                ((4 * D) - (2 * F)),
                ((6 * D) + (2 * F)),
                ((3 * D) + M), 
                ((5 * D) + M),
                (13 * D),
                ((20 * D) - M), 
                ((3 * D) + ( 2 *M)),
                ((4 * D) + (2 * F) - (2 *M)), 
                ((D + (2*M))), 
                ((22 * D) - M),
                (4 *F),
                ((6 * D) - (2 *F)),
                ((2 * D) - (2 * F) + M), 
                (2 * M), 
                ((2 *F) - M),
                ((2 * D) + (4 * F)), 
                ((2 * F) - (2 * M)),
                ((2 * D) - (2 * F) +(2 * M)),
                (24 * D),
                ((4 * D) - (4 * F)),
                ((2 * D) + (2 * M)),
                (D - M)
            ]
        }

        # Calculate sine values and final value
        T50_sin_values = [math.sin(math.radians(arg)) for arg in T50["Argument of Sine"]]
        data = pd.DataFrame(T50)
        data["sine_values"] = T50_sin_values
        data['values'] = data['sine_values'] * data['Coefficient']
        sum_of_values = data['values'].sum()
        
        # Final JDE
        JDE_final = JDE_initial + sum_of_values
        
        return JDE_final

# Function to convert Julian Date to Gregorian date
def julian_to_gregorian(jd):
    jd = jd + 0.5
    z = int(jd)
    f = jd - z
    if z < 2299161:
        A = z
    else:
        alpha = int((z - 1867216.25) / 36524.25)
        A = z + 1 + alpha - int(alpha / 4)

    B = A + 1524
    C = int((B - 122.1) / 365.25)
    D = int(365.25 * C)
    E = int((B - D) / 30.6001)

    day = B - D - int(30.6001 * E) + f
    if E < 14:
        month = E - 1
    else:
        month = E - 13

    if month > 2:
        year = C - 4716
    else:
        year = C - 4715

    # Calculate the full day and time fraction
    # Calculates day time
    day_fraction = day % 1
    day = int(day)
    
    # Convert the fractional day into hours, minutes, and seconds
    total_seconds = day_fraction * 86400  # 24 * 60 * 60 seconds in a day
    hour = int(total_seconds // 3600)
    # Finds remaining minutes by remainder
    minute = int((total_seconds % 3600) // 60)
    #Finds remaining seconds by remainder
    second = int(total_seconds % 60)

    return datetime(year, month, day, hour, minute, second)

# List to store formatted Gregorian dates for each valid k value
formatted_dates = []

# Loop through sorted k values
for k in sorted_list:
    # Calculate JDE for the current k value
    jde_value = calculate_jde(k)
    
    # Check if JDE calculation was successful (i.e., not None)
    if jde_value is not None:
        # Convert JDE to Gregorian date
        gregorian_date = julian_to_gregorian(jde_value)
        # Format the date to a readable string format
        formatted_date = gregorian_date.strftime("%Y-%m-%d %H:%M:%S")
        formatted_dates.append(formatted_date)  # Store the formatted date




# Create a DataFrame from the formatted dates list
dates_df = pd.DataFrame(formatted_dates, columns=["Dates"])
dates_df = dates_df.drop_duplicates()

dates_df

# Code based on Jean Meeus' Astronomical Algorithms Chapter 7 (Julian Day), Chapter 50 (Perigee and Apogee of the Moon), and Chapter 10 (Dynamical and Universal Time)
# Accurate to date, hour and within 1 or 2 minutes of actual value
# Final times are given in UT, which is within .9 seconds of UTC and EST has an offset of 5 hours from UT (https://eclipse.gsfc.nasa.gov/SEhelp/TimeZone.html)

,Dates
0,2025-01-07 23:35:42
2,2025-02-02 02:44:08
3,2025-03-01 21:19:39
4,2025-03-30 05:27:24
5,2025-04-27 16:16:01
7,2025-05-26 01:37:54
8,2025-06-23 04:44:11
9,2025-07-20 13:53:41
11,2025-08-14 18:02:28
12,2025-09-10 12:10:42


### Apogee Calculations (for checking)

In [43]:
import pandas as pd
import time
from datetime import datetime, timedelta
import math

# Year and k calculation as per the initial example
year = 2025.0417
k = (year - 1999.97) * 13.2555

# Round k to the nearest integer or 0.5 as per the earlier logic
if abs(k - round(k)) < abs(k - round(k * 2) / 2):
    k = round(k)  # Round to nearest integer
else:
    k = round(k * 2) / 2  # Round to nearest 0.5

# Calculate T
T = (k / 1325.55)

# Calculate the initial JDE
JDE_initial = 2451534.6698 + (27.55454989 * k) + (-0.0006691 * (T ** 2)) + (-0.000001098 * (T ** 3)) + (0.0000000052 * (T ** 4))

D = 171.9179 + (335.9106046 * k) - (.0100383 * T**2) - (.00001156 * T**3) + (.000000055 * T**4)
M = 347.3477 + (27.1577721 * k) - (.0008130 * T**2) - (.0000010 * T**3)
F = 316.6109 + (364.5287911 * k) - (.0125053 * T**2) - (.0000148 * T**3)


# Define the coefficient values, including those depending on T
T50 = {
    "Coefficient": [
        0.4392,
        0.0684,
        (0.0456 - (0.00011 * T)),
        (0.0426 - (0.00011 * T)),
        0.0212,
        -0.0189,
        0.0144,
        0.0113,
        0.0047,
        0.0036,
        0.0035,
        0.0034,
        -0.0034,
        0.0022,
        -0.0017,
        0.0013,
        0.0011,
        0.0010,
        0.0009,
        0.0007,
        0.0006,
        0.0005,
        0.0005,
        0.0004,
        0.0004,
        0.0004,
        -0.0004,
        -0.0004,
        0.0003,
        0.0003,
        0.0003,
        -0.0003
    ],
    "Argument of Sine": [
        (2 * D),
        (4 * D),
        (M), 
        ((2 * D) - M),
        (2 * F), 
        (D), 
        (6 * D),
        ((4 * D) - M), 
        ((2 * D) + (2 * F)),
        (D + M),
        (8 * D),
        ((6 * D) - M),
        ((2 * D) - (2 * F)),
        ((2 * D) - (2 * M)),
        (3 * D),
        ((4 * D) + (2 * F)),
        ((8 * D) - M),
        ((4 * D) - (2 * M)),
        (10 * D), 
        ((3 * D) + M),
        (2 * M),
        ((2 * D) + M),
        ((2 * D) + (2 * M)),
        ((6 * D) + (2 * F)),
        ((6 * D) - (2 * M)),
        ((10 * D) - M),
        (5 * D), 
        ((4 * D) - (2 * F)),
        ((2 * F) + M),
        (12 * D),
        ((2 * D) + (2 * F) - M), 
        (D - M)
    ]
}

# Apply the sine function to all the arguments
T50_sin_values = [math.sin(math.radians(arg)) for arg in T50["Argument of Sine"]]

# Create DataFrame and add the sine values
data = pd.DataFrame(T50)
data["sine_values"] = T50_sin_values

# Multiply sine values with the coefficients
data['values'] = data['sine_values'] * data['Coefficient']

# Calculate the sum of values
sum_of_values = data['values'].sum()

# Add coefficient sum to JDE
JDE_final = (JDE_initial + sum_of_values)

# Function to convert Julian Date to Gregorian date
def julian_to_gregorian(jd):
    jd = jd + 0.5
    z = int(jd)
    f = jd - z
    if z < 2299161:
        A = z
    else:
        alpha = int((z - 1867216.25) / 36524.25)
        A = z + 1 + alpha - int(alpha / 4)

    B = A + 1524
    C = int((B - 122.1) / 365.25)
    D = int(365.25 * C)
    E = int((B - D) / 30.6001)

    day = B - D - int(30.6001 * E) + f
    if E < 14:
        month = E - 1
    else:
        month = E - 13

    if month > 2:
        year = C - 4716
    else:
        year = C - 4715

    # Calculate the full day and time fraction
    # Calculates day time
    day_fraction = day % 1
    day = int(day)
    
    # Convert the fractional day into hours, minutes, and seconds
    total_seconds = day_fraction * 86400  # 24 * 60 * 60 seconds in a day
    hour = int(total_seconds // 3600)
    # Finds remaining minutes by remainder
    minute = int((total_seconds % 3600) // 60)
    #Finds remaining seconds by remainder
    second = int(total_seconds % 60)

    return datetime(year, month, day, hour, minute, second)

# Example usage
gregorian_date = julian_to_gregorian(JDE_final)
print("Gregorian Date:", gregorian_date)

# Code based on Jean Meeus' Astronomical Algorithms Chapter 7 (Julian Day), Chapter 50 (Perigee and Apogee of the Moon), and Chapter 10 (Dynamical and Universal Time)
# Accurate to date, hour and minute
# Final times are given in UT, which is within .9 seconds of UTC and EST has an offset of 5 hours from UT (https://eclipse.gsfc.nasa.gov/SEhelp/TimeZone.html)


Gregorian Date: 2025-01-21 04:56:18


### Working Code (testing)

In [101]:
# Load the CSV file into a DataFrame (make sure the CSV is correctly formatted)
t_values_df = pd.read_csv(r"C:\Users\pawoo\OneDrive\Documents\Coastal_Hazards_Lab_Stuff\NCKT\NCKT_tabulated_T_values.csv")
t_values_df.columns = t_values_df.columns.str.strip()

# User input year
input_year = 2025

# Adjust t1, t2, and t3 with the input year
t_values_df['t1 + year'] = t_values_df['t1'] + input_year
t_values_df['t2 + year'] = t_values_df['t2'] + input_year
t_values_df['t3 + year'] = t_values_df['t3'] + input_year


t_values_df

# Function to calculate and filter k values
def process_k_values(t_plus_year):
    # Step 1: Calculate initial k
    k = (t_plus_year - 1999.97) * 13.2555

    # Step 2: Round k to nearest integer or 0.5 as per logic
    if abs(k - round(k)) < abs(k - round(k * 2) / 2):
        k = round(k)  # Round to nearest integer
    else:
        k = round(k * 2) / 2  # Round to nearest 0.5
    
    # Step 3: Remove k values ending in .5
    if k % 1 == 0.5:
        return None  # Filter out .5 values
    else:
        return k

# Iterate over each t + year column, process each month, and print results
for col in ['t1 + year', 't2 + year', 't3 + year']:
    t_values_df[f'k_{col}'] =  t_values_df[col].apply(process_k_values)  # Apply function to calculate and filter k

# Select only the columns that contain 'k' values (created earlier in your DataFrame)
k_columns = [f'k_{col}' for col in ['t1 + year', 't2 + year', 't3 + year']]

# Concatenate all the k columns into a single Series, automatically dropping NaNs
k_values_series = pd.concat([t_values_df[col].dropna() for col in k_columns])

# Convert the concatenated Series into a list
k_values_list = k_values_series.tolist()

sorted_list = sorted(k_values_list)

print(sorted_list)


# Function to calculate JDE for a given k value
def calculate_jde(k):
        
        JDE_initial = 2451534.6698 + (27.55454989 * k) + (-0.0006691 * (T ** 2)) + (-0.000001098 * (T ** 3)) + (0.0000000052 * (T ** 4))

        D = 171.9179 + (335.9106046 * k) - (0.0100383 * T**2) - (0.00001156 * T**3) + (0.000000055 * T**4)
        M = 347.3477 + (27.1577721 * k) - (0.0008130 * T**2) - (0.0000010 * T**3)
        F = 316.6109 + (364.5287911 * k) - (0.0125053 * T**2) - (0.0000148 * T**3)

        # Define the coefficients
        ## Perigee terms: 
        T50 = {
            "Coefficient": [
            -1.6769,
            0.4589,
            -0.1856,
            0.0883,
            (-0.0773 + (0.00019 * T)),
            (0.0502 - (0.00013 * T)),
            -0.0460,
            (0.0422 - (0.00011 * T)),
            -0.0256,
            0.0253,
            0.0237,
            0.0162,
            -0.0145,
            0.0129,
            -0.0112,
            -0.0104,
            0.0086,
            0.0069,
            0.0066,
            -0.0053,
            -0.0052,
            -0.0046,
            -0.0041,
            0.0040,
            0.0032,
            -0.0032,
            0.0031,
            -0.0029,
            0.0027,
            0.0027,
            -0.0027,
            0.0024,
            -0.0021,
            -0.0021,
            -0.0021,
            0.0019,
            -0.0018,
            -0.0014,
            -0.0014,
            -0.0014,
            0.0014,
            -0.0014,
            0.0013,
            0.0013,
            0.0011,
            -0.0011,
            -0.0010,
            -0.0009,
            -0.0008,
            0.0008,
            0.0008,
            0.0007,
            0.0007,
            0.0007,
            -0.0006,
            -0.0006,
            0.0006,
            0.0005,
            0.0005,
            -0.0004,
            ],

            "Argument of Sine": [
                (2 * D),
                (4 * D),
                (6 *D),
                (8 * D),
                ((2 * D) - M),
                (M),
                (10* D),
                ((4 * D) - M), 
                ((6*D) - M),
                (12 * D), 
                (D), 
                ((8 * D) - M),
                (14 * D), 
                (2 * F), 
                (3 *D), 
                ((10 * D) - M),
                (16 * D),
                ((12 * D) - M),
                (5 * D), 
                ((2 * D) + (2 * F)),
                (18 * D), 
                ((14 * D) - M), 
                (7 *D), 
                ((2 * D) + M), 
                (20 * D), 
                (D + M), 
                ((16 * D) - M),
                ((4 * D) + M),
                ( 9 * M),
                ((4 * D )+ (2 * F)),
                ((2 * D) - (2 * M)),
                ((4 * D) - (2 * M)),
                ((6 * D) - (2 * M)),
                (22 * D), 
                ((18 * D) - (M)),
                ((6*D) + M), 
                (11 * D),
                ((8 * D) + M),
                ((4 * D) - (2 * F)),
                ((6 * D) + (2 * F)),
                ((3 * D) + M), 
                ((5 * D) + M),
                (13 * D),
                ((20 * D) - M), 
                ((3 * D) + ( 2 *M)),
                ((4 * D) + (2 * F) - (2 *M)), 
                ((D + (2*M))), 
                ((22 * D) - M),
                (4 *F),
                ((6 * D) - (2 *F)),
                ((2 * D) - (2 * F) + M), 
                (2 * M), 
                ((2 *F) - M),
                ((2 * D) + (4 * F)), 
                ((2 * F) - (2 * M)),
                ((2 * D) - (2 * F) +(2 * M)),
                (24 * D),
                ((4 * D) - (4 * F)),
                ((2 * D) + (2 * M)),
                (D - M)
            ]
        }

        # Calculate sine values and final value
        T50_sin_values = [math.sin(math.radians(arg)) for arg in T50["Argument of Sine"]]
        data = pd.DataFrame(T50)
        data["sine_values"] = T50_sin_values
        data['values'] = data['sine_values'] * data['Coefficient']
        sum_of_values = data['values'].sum()
        
        # Final JDE
        JDE_final = JDE_initial + sum_of_values
        
        return JDE_final

# Function to convert Julian Date to Gregorian date
def julian_to_gregorian(jd):
    jd = jd + 0.5
    z = int(jd)
    f = jd - z
    if z < 2299161:
        A = z
    else:
        alpha = int((z - 1867216.25) / 36524.25)
        A = z + 1 + alpha - int(alpha / 4)

    B = A + 1524
    C = int((B - 122.1) / 365.25)
    D = int(365.25 * C)
    E = int((B - D) / 30.6001)

    day = B - D - int(30.6001 * E) + f
    if E < 14:
        month = E - 1
    else:
        month = E - 13

    if month > 2:
        year = C - 4716
    else:
        year = C - 4715

    # Calculate the full day and time fraction
    # Calculates day time
    day_fraction = day % 1
    day = int(day)
    
    # Convert the fractional day into hours, minutes, and seconds
    total_seconds = day_fraction * 86400  # 24 * 60 * 60 seconds in a day
    hour = int(total_seconds // 3600)
    # Finds remaining minutes by remainder
    minute = int((total_seconds % 3600) // 60)
    #Finds remaining seconds by remainder
    second = int(total_seconds % 60)

    return datetime(year, month, day, hour, minute, second)

# List to store formatted Gregorian dates for each valid k value
formatted_dates = []

# Loop through sorted k values
for k in sorted_list:
    # Calculate JDE for the current k value
    jde_value = calculate_jde(k)
    
    # Check if JDE calculation was successful (i.e., not None)
    if jde_value is not None:
        # Convert JDE to Gregorian date
        gregorian_date = julian_to_gregorian(jde_value)
        # Format the date to a readable string format
        formatted_date = gregorian_date.strftime("%Y-%m-%d %H:%M:%S")
        formatted_dates.append(formatted_date)  # Store the formatted date

# Display the resulting formatted Gregorian dates
print("Formatted Gregorian Dates:", formatted_dates)


# Create a DataFrame from the formatted dates list
dates_df = pd.DataFrame(formatted_dates, columns=["Dates"])

dates_df

[332.0, 332.0, 333.0, 334.0, 335.0, 336.0, 336.0, 337.0, 338.0, 339.0, 339.0, 340.0, 341.0, 342.0, 343.0, 343.0, 344.0]
Formatted Gregorian Dates: ['2025-01-07 23:34:48', '2025-01-07 23:34:48', '2025-02-02 02:44:10', '2025-03-01 21:17:39', '2025-03-30 05:31:25', '2025-04-27 16:13:40', '2025-04-27 16:13:40', '2025-05-26 01:34:59', '2025-06-23 04:51:26', '2025-07-20 13:47:29', '2025-07-20 13:47:29', '2025-08-14 18:02:48', '2025-09-10 12:15:57', '2025-10-08 12:30:48', '2025-11-05 22:32:30', '2025-11-05 22:32:30', '2025-12-04 11:08:13']


,Dates
0,2025-01-07 23:34:48
1,2025-01-07 23:34:48
2,2025-02-02 02:44:10
3,2025-03-01 21:17:39
4,2025-03-30 05:31:25
5,2025-04-27 16:13:40
6,2025-04-27 16:13:40
7,2025-05-26 01:34:59
8,2025-06-23 04:51:26
9,2025-07-20 13:47:29


In [ ]:
#Table 50.A Calculations 

# Enter year here in form "year.month" where month is in fraction of year
# Ex. The end of April in 2041 would be 2041.33


import pandas as pd
import time
from datetime import datetime, timedelta
import math

# Year and k calculation as per the initial example
year = 2025.0
k = (year - 1999.97) * 13.2555


# Round k to the nearest integer or 0.5 as per the earlier logic
if abs(k - round(k)) < abs(k - round(k * 2) / 2):
    k = round(k)  # Round to nearest integer
else:
    k = round(k * 2) / 2  # Round to nearest 0.5

print(k)

# Calculate T
T = (k / 1325.55)

# Calculate the initial JDE
JDE_initial = 2451534.6698 + (27.55454989 * k) + (-0.0006691 * (T ** 2)) + (-0.000001098 * (T ** 3)) + (0.0000000052 * (T ** 4))

D = 171.9179 + (335.9106046 * k) - (.0100383 * T**2) - (.00001156 * T**3) + (.000000055 * T**4)
M = 347.3477 + (27.1577721 * k) - (.0008130 * T**2) - (.0000010 * T**3)
F = 316.6109 + (364.5287911 * k) - (.0125053 * T**2) - (.0000148 * T**3)
year = 1988.75


## Perigee terms: 
T50 = {
    "Coefficient": [
    -1.6769,
    0.4589,
    -0.1856,
    0.0883,
    (-0.0773 + (0.00019 * T)),
    (0.0502 - (0.00013 * T)),
    -0.0460,
    (0.0422 - (0.00011 * T)),
    -0.0256,
    0.0253,
    0.0237,
    0.0162,
    -0.0145,
    0.0129,
    -0.0112,
    -0.0104,
    0.0086,
    0.0069,
    0.0066,
    -0.0053,
    -0.0052,
    -0.0046,
    -0.0041,
    0.0040,
    0.0032,
    -0.0032,
    0.0031,
    -0.0029,
    0.0027,
    0.0027,
    -0.0027,
    0.0024,
    -0.0021,
    -0.0021,
    -0.0021,
    0.0019,
    -0.0018,
    -0.0014,
    -0.0014,
    -0.0014,
    0.0014,
    -0.0014,
    0.0013,
    0.0013,
    0.0011,
    -0.0011,
    -0.0010,
    -0.0009,
    -0.0008,
    0.0008,
    0.0008,
    0.0007,
    0.0007,
    0.0007,
    -0.0006,
    -0.0006,
    0.0006,
    0.0005,
    0.0005,
    -0.0004,
    ],

    "Argument of Sine": [
        (2 * D),
        (4 * D),
        (6 *D),
        (8 * D),
        ((2 * D) - M),
        (M),
        (10* D),
        ((4 * D) - M), 
        ((6*D) - M),
        (12 * D), 
        (D), 
        ((8 * D) - M),
        (14 * D), 
        (2 * F), 
        (3 *D), 
        ((10 * D) - M),
        (16 * D),
        ((12 * D) - M),
        (5 * D), 
        ((2 * D) + (2 * F)),
        (18 * D), 
        ((14 * D) - M), 
        (7 *D), 
        ((2 * D) + M), 
        (20 * D), 
        (D + M), 
        ((16 * D) - M),
        ((4 * D) + M),
        ( 9 * M),
        ((4 * D )+ (2 * F)),
        ((2 * D) - (2 * M)),
        ((4 * D) - (2 * M)),
        ((6 * D) - (2 * M)),
        (22 * D), 
        ((18 * D) - (M)),
        ((6*D) + M), 
        (11 * D),
        ((8 * D) + M),
        ((4 * D) - (2 * F)),
        ((6 * D) + (2 * F)),
        ((3 * D) + M), 
        ((5 * D) + M),
        (13 * D),
        ((20 * D) - M), 
        ((3 * D) + ( 2 *M)),
        ((4 * D) + (2 * F) - (2 *M)), 
        ((D + (2*M))), 
        ((22 * D) - M),
        (4 *F),
        ((6 * D) - (2 *F)),
        ((2 * D) - (2 * F) + M), 
        (2 * M), 
        ((2 *F) - M),
        ((2 * D) + (4 * F)), 
        ((2 * F) - (2 * M)),
        ((2 * D) - (2 * F) +(2 * M)),
        (24 * D),
        ((4 * D) - (4 * F)),
        ((2 * D) + (2 * M)),
        (D - M)
    ]
}

# Apply the sine function to all the arguments
T50_sin_values = [math.sin(math.radians(arg)) for arg in T50["Argument of Sine"]]

# Create DataFrame and add the sine values
data = pd.DataFrame(T50)
data["sine_values"] = T50_sin_values

# Multiply sine values with the coefficients
data['values'] = data['sine_values'] * data['Coefficient']

# Calculate the sum of values
sum_of_values = data['values'].sum()

# Add coefficient sum to JDE
JDE_final = (JDE_initial + sum_of_values)

# Function to convert Julian Date to Gregorian date
def julian_to_gregorian(jd):
    jd = jd + 0.5
    z = int(jd)
    f = jd - z
    if z < 2299161:
        A = z
    else:
        alpha = int((z - 1867216.25) / 36524.25)
        A = z + 1 + alpha - int(alpha / 4)

    B = A + 1524
    C = int((B - 122.1) / 365.25)
    D = int(365.25 * C)
    E = int((B - D) / 30.6001)

    day = B - D - int(30.6001 * E) + f
    if E < 14:
        month = E - 1
    else:
        month = E - 13

    if month > 2:
        year = C - 4716
    else:
        year = C - 4715

    # Calculate the full day and time fraction
    # Calculates day time
    day_fraction = day % 1
    day = int(day)
    
    # Convert the fractional day into hours, minutes, and seconds
    total_seconds = day_fraction * 86400  # 24 * 60 * 60 seconds in a day
    hour = int(total_seconds // 3600)
    # Finds remaining minutes by remainder
    minute = int((total_seconds % 3600) // 60)
    #Finds remaining seconds by remainder
    second = int(total_seconds % 60)

    return datetime(year, month, day, hour, minute, second)

# Example usage
gregorian_date = julian_to_gregorian(JDE_final)
print("Gregorian Date:", gregorian_date)

# Code based on Jean Meeus' Astronomical Algorithms Chapter 7 (Julian Day), Chapter 50 (Perigee and Apogee of the Moon), and Chapter 10 (Dynamical and Universal Time)
# Accurate to date, hour and minute
# Final times are given in UT, which is within .9 seconds of UTC and EST has an offset of 5 hours from UT (https://eclipse.gsfc.nasa.gov/SEhelp/TimeZone.html)